In [ ]:
import sys
import os
sys.path.append('util/')
import importlib

import roi_util_scripts as roi_util
import pickle
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

In [ ]:
import spectral

#fname = 'data/morven_4000/raw_4000_or_ref.img'
#fname_hdr = 'data/morven_4000/raw_4000_or_ref.hdr'

fname = 'data/Greenhead_aug/raw_39771_or_ref.img'
fname_hdr = 'data/Greenhead_aug/raw_39771_or_ref.hdr'

#fname = 'data/5-8-2025/100133_Allied_05_08_2025_2015_06_04_17_50_15/raw_0_ref'
#fname_hdr = 'data/5-8-2025/100133_Allied_05_08_2025_2015_06_04_17_50_15/raw_0_ref.hdr'

# Open the image and read into an array
im = spectral.envi.open(fname_hdr, fname)
wl_img = np.asarray(im.bands.centers)

In [ ]:
# data lists

d_spectra = []
d_plant = []
d_part = []
d_health = []
d_age = []
d_lifecycle = []

d_dir = []

yd_all_dict = {
    'plant': d_plant,
    'age': d_age,
    'part': d_part,
    'health': d_health,
    'lifecycle': d_lifecycle
}

code_category_dict = {
    'plant': roi_util.plant_codes,
    'age': roi_util.age_codes,
    'part': roi_util.principal_part_codes,
    'health': roi_util.health_codes,
    'lifecycle': roi_util.lifecycle_codes
}

In [ ]:
## Find ROI data

roi_files = roi_util.find_roi_files('data/pkl/rois_labeled')

print(f"Number of ROI files found: {len(roi_files)}")

In [ ]:
## Prepare ROI data

resample_rois = True

for roi_filename in roi_files:
    print(roi_filename)

   # Unpickling the dictionary
    with open(roi_filename, 'rb') as f:
        roiData = pickle.load(f)
        roi_df = roiData.df # a DataFrame holding all the data for the ROI

    roi_spectra = roi_df.to_numpy()[:,4:]
    roi_spectra = roi_spectra.astype(np.float32)

    if resample_rois:
        wl_roi = roi_df.columns.to_numpy()[4:]
        resampler_roi = spectral.BandResampler(wl_roi, wl_img)
        roi_spectra_resampled = resampler_roi(roi_spectra.T).T
        roi_spectra = roi_spectra_resampled

    roi_spectra_names = roi_df['Name'].to_numpy()

    roi_names = roiData.names # the names of the ROIs

    print(f"Number of ROIs found in {roi_filename}: {len(roi_names)}")
    print(roi_names)

    for name in roi_names:
        roi_class_spectra = roi_spectra[roi_spectra_names==name]

        if name == 'Genus_spe_N_N_N_N':
            new_name = 'Genus_spe_MX_N_N_N'
            print(f"Renaming {name} to {new_name}")
            name = new_name

        if name[-1] != '_':
            name = name + '_'

        # parse name for metadata
        class_data_dict = {}          
        
        for cat, codes in code_category_dict.items():
            class_data_dict[cat] = 'N'
            for key, value in codes.items():
                if cat == 'plant':
                    if (name[:8].lower()==key.lower()) or (name[:9].lower()==key.lower()):
                        class_data_dict[cat] = value[0] + '_' + value[1]
                else:
                    if '_'+key+'_' in name:
                        class_data_dict[cat] = value[1]
                        #print(key, class_data_dict[cat])
                        #print(class_data_dict)
            
            #if class_data_dict[cat] == 'N':
            #    print(cat, name)

        #print(class_data_dict)

        # for each spectrum in class_spectra, append to each list
        for roi_spectrum in roi_class_spectra:

            d_spectra.append(roi_spectrum)
            
            for key in yd_all_dict:
                #print(key)
                #print(len(yd_all_dict[key]))
                #print(yd_all_dict[key])
                yd_all_dict[key].append(class_data_dict[key])
                #print(len(yd_all_dict[key]))
            
            dir = os.path.dirname(roi_filename)
            d_dir.append(dir.rsplit('/', 1)[1])

In [ ]:
d_spectra = np.asarray(d_spectra)
print(d_spectra.shape)

for key in yd_all_dict:
    yd_all_dict[key] = np.asarray(yd_all_dict[key])
    print(key, yd_all_dict[key].shape)

In [ ]:
d_dir = np.asarray(d_dir)

In [ ]:
print(d_dir)

In [ ]:
dir_str = "eval"

selected_indices = np.where(d_dir == dir_str)[0]
#print(selected_indices)

plt.figure(figsize=(12,6))
plt.grid(True)
for idx in selected_indices:
    plt.plot(wl_img, d_spectra[idx], lw=1, color='b', alpha=0.05)
mean = np.mean(d_spectra[selected_indices], axis=0)
plt.plot(wl_img, mean, color='r')
plt.xlabel('Wavelength')
plt.ylabel('Refletance')
plt.title(f'Spectra: {dir_str}')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncols=3)


dir_str = "greenhead"

selected_indices = np.where((d_dir == dir_str) & (yd_all_dict['plant'] == "Iva_frutescens"))[0]
#print(selected_indices)

plt.figure(figsize=(12,6))
plt.grid(True)
for idx in selected_indices:
    plt.plot(wl_img, d_spectra[idx], lw=1, color='b', alpha=0.05)
mean = np.mean(d_spectra[selected_indices], axis=0)
plt.plot(wl_img, mean, color='r')
plt.xlabel('Wavelength')
plt.ylabel('Refletance')
plt.title(f'Spectra: {dir_str} and Iva')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncols=3)

dir_str = "microscene_temp"

selected_indices = np.where((d_dir == dir_str) & (yd_all_dict['plant'] == "Iva_frutescens"))[0]
#print(selected_indices)

plt.figure(figsize=(12,6))
plt.grid(True)
for idx in selected_indices:
    plt.plot(wl_img, d_spectra[idx], lw=1, color='b', alpha=0.05)
mean = np.mean(d_spectra[selected_indices], axis=0)
plt.plot(wl_img, mean, color='r')
plt.xlabel('Wavelength')
plt.ylabel('Refletance')
plt.title(f'Spectra: {dir_str} and Iva')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncols=3)

dir_str = "microscene_temp"

selected_indices = np.where((d_dir == dir_str) & (yd_all_dict['plant'] == "Baccharis_halimifolia"))[0]
#print(selected_indices)

plt.figure(figsize=(12,6))
plt.grid(True)
for idx in selected_indices:
    plt.plot(wl_img, d_spectra[idx], lw=1, color='b', alpha=0.05)
mean = np.mean(d_spectra[selected_indices], axis=0)
plt.plot(wl_img, mean, color='r')
plt.xlabel('Wavelength')
plt.ylabel('Refletance')
plt.title(f'Spectra: {dir_str} and Bacc')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncols=3)

In [ ]:
import sys
sys.path.append('util/')
import importlib
import pandas as pd

import util_scripts as util

import os
from dotenv import load_dotenv

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

_ = load_dotenv()

MONGO_DBR_URI = os.getenv('MONGO_DBR_URI')

In [ ]:
reload_data_driver = False

if (reload_data_driver):

    records = []

    # Create a new client and connect to the server
    client = MongoClient(MONGO_DBR_URI, server_api=ServerApi('1'))

    # Send a ping to confirm a successful connection
    try:
        client.admin.command('ping')
        print("Pinged your deployment. You successfully connected to MongoDB!")
    except Exception as e:
        print(e)

    db = client["upwins_db"]
    view_name = "spectral_library"
    spectral_library = db[view_name]

    records = spectral_library.find()

    df = pd.DataFrame(records)
    df.to_pickle('data/pkl/library_with_Genus_species.pkl')

else:
    df = pd.read_pickle('data/pkl/library_with_Genus_species.pkl')
    #df = pd.read_pickle('data/pkl/library.pkl')

In [ ]:
sc = util.SpectralCollection(df)
wl_lib = sc.wl
name = sc.name
spectra = sc.spectra

In [ ]:
# Create a BandResampler, which is a function that resamples spectra from one source to match a different source.
# See: https://www.spectralpython.net/class_func_ref.html?highlight=resampling#spectral.algorithms.resampling.BandResampler
# Inputs: the first input is the wavelengths for the spectra that you are going to resample, the second input is the wavelengths that you want to resample to.
resampler = spectral.BandResampler(wl_lib, wl_img)
spectra_resampled = resampler(spectra.T).T

print(f'The shape of the spectral library is {spectra_resampled.shape}.')
print(f'({spectra_resampled.shape[0]} spectra with {spectra_resampled.shape[1]} bands.)')

In [ ]:
# Skip the next two cells unless there is a need to filter inputs

filter = {
    'name': 'Iva_frutescens',
    #'fname': '',
    #'genus': '',
    #'species': '',
    #'age': 'M',
    #'health': 'H',
    #'part': 'L',
    #'type': '',
    #'lifecycle': '',
    #'date': ''
}

#prediction_class = 'age'

selected_indices = sc.select_indicies_with_filter(filter)

nSpec = len(selected_indices)
print(f"Number of Spectra: {nSpec}")

In [ ]:
means = {}
class_spectra = spectra_resampled[selected_indices]
class_nSpec = class_spectra.shape[0]

plt.figure(figsize=(10,6)) 
for i in range(class_nSpec):
    plt.plot(wl_img,class_spectra[i,:].flatten(), color='b', alpha=0.05)
means[filter['name']] = np.mean(class_spectra, axis=0)
plt.plot(wl_img, means[filter['name']], color='r')
plt.minorticks_on()
plt.grid(True, which='major', c='k')
plt.grid(True, which='minor', linewidth=0.25)
plt.title(f'Spectra for {filter['name']}')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.grid(True)

In [ ]:
# Skip the next two cells unless there is a need to filter inputs

filter = {
    'name': 'Baccharis_halimifolia',
    #'fname': '',
    #'genus': '',
    #'species': '',
    #'age': 'M',
    #'health': 'H',
    #'part': 'L',
    #'type': '',
    #'lifecycle': '',
    #'date': ''
}

#prediction_class = 'age'

selected_indices = sc.select_indicies_with_filter(filter)

nSpec = len(selected_indices)
print(f"Number of Spectra: {nSpec}")

In [ ]:
means = {}
class_spectra = spectra_resampled[selected_indices]
class_nSpec = class_spectra.shape[0]

plt.figure(figsize=(10,6)) 
for i in range(class_nSpec):
    plt.plot(wl_img,class_spectra[i,:].flatten(), color='b', alpha=0.05)
means[filter['name']] = np.mean(class_spectra, axis=0)
plt.plot(wl_img, means[filter['name']], color='r')
plt.minorticks_on()
plt.grid(True, which='major', c='k')
plt.grid(True, which='minor', linewidth=0.25)
plt.title(f'Spectra for {filter['name']}')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.grid(True)

In [ ]:
# Skip the next two cells unless there is a need to filter inputs

filter = {
    'name': 'Rosa_rugosa',
    #'fname': '',
    #'genus': '',
    #'species': '',
    #'age': 'M',
    #'health': 'H',
    #'part': 'L',
    #'type': '',
    #'lifecycle': '',
    #'date': ''
}

#prediction_class = 'age'

selected_indices = sc.select_indicies_with_filter(filter)

nSpec = len(selected_indices)
print(f"Number of Spectra: {nSpec}")

In [ ]:
means = {}
class_spectra = spectra_resampled[selected_indices]
class_nSpec = class_spectra.shape[0]

plt.figure(figsize=(10,6)) 
for i in range(class_nSpec):
    plt.plot(wl_img,class_spectra[i,:].flatten(), color='b', alpha=0.05)
means[filter['name']] = np.mean(class_spectra, axis=0)
plt.plot(wl_img, means[filter['name']], color='r')
plt.minorticks_on()
plt.grid(True, which='major', c='k')
plt.grid(True, which='minor', linewidth=0.25)
plt.title(f'Spectra for {filter['name']}')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.grid(True)